# 将Python脚本转换为GRASS GIS模块

* 首先，每个Python脚本中必须包含 *[shebang](https://en.wikipedia.org/wiki/Shebang_%28Unix%29)* 放在第一行定义解释器，在这里使用`＃！/ usr / bin / env`。 

* 其次，代码应作为函数编写，而不是直接在顶层编写，标准模板如下。但是，请注意，IPython Notebook中的第一行是`%% python`，它告诉IPython将单元内容作为Python脚本执行。 稍后，我们将以更接近GRASS GIS中脚本调用方式的方式重写脚本调用。



In [1]:
%%python
#!/usr/bin/env python

def main():
    pass

if __name__ == '__main__':
    main()


`if __name__ == '__main__':`的构造是Python中标记代码的标准方式，当文件用作脚本时应执行该代码，最佳做法是使用上述模板。

## GRASS GIS 解析器


每个GRASS GIS模块都必须使用GRASS解析器机制。 这个非常高级的解析器可以帮助检查用户输入，格式化帮助文本并有选择地为新模块创建图形用户界面。

在Python中，这意味着从`grass.script`包中调用`parser（）`函数。 此函数解析在Python文件开头（在“ shebang”下面）写的特殊注释，在使用模块时处理命令行中提供的参数，并在模块内相应地提供这些数据。 这些特殊注释以`＃％`开头，可以称为*“接口定义注释”*。



### 简化模板



*接口定义注释*至少应包含模块说明和两个关键字，如下所示。

[Existing GRASS GIS Python scripts](https://trac.osgeo.org/grass/browser/grass/branches/releasebranch_7_0/scripts/)，可能有助于了解最佳做法，这些值在`module`部分中定义，其中包含`description`和`keyword`关键字。


In [7]:
%%file r.viewshed.points.py
#!/usr/bin/env python

#%module
#% description: Compute and analyze viewsheds
#% keyword: raster
#% keyword: viewshed
#%end

import grass.script as gscript

def main():
    gscript.parser()

if __name__ == '__main__':
    main()



Overwriting r.viewshed.points.py


请注意，GRASS GIS解析器在文件末尾需要换行符，在这种情况下，我们使用两个空行，因为IPython Notebook将删除其中的一个。

要运行脚本，我们需要将其权限设置为'executable'，我们使用`chmod`命令。

In [10]:
!chmod u+x r.viewshed.points.py #! 表示在命令行运行

现在，可以使用`--help`将脚本作为GRASS GIS模块执行，以获取命令行帮助输出。

In [11]:
!./r.viewshed.points.py --help

Compute and analyze viewsheds

Usage:
 r.viewshed.points.py [--help] [--verbose] [--quiet] [--ui]

Flags:


### 添加输入参数

现在，为模块添加参数，在*界面定义注释*中，我们为每个注释中添加了一个 `option` 部分，分别定义了标准（或预定义）选项。对于高程栅格图层，我们使用`G_OPT_R_INPUT`，对于点矢量地图，使用`G_OPT_V_INPUT`。这两个标准选项默认情况下都命名为`input`，因此我们必须给它们指定唯一的名称。在这里，我们将对栅格使用`elevation`，对矢量地图使用`points`，该选项的名称是使用名为`key`的键定义的。


In [18]:
%%file r.viewshed.points.py
#!/usr/bin/env python

#%module
#% description: Compute and analyze viewsheds
#% keyword: raster
#% keyword: viewshed
#%end
#%option G_OPT_R_INPUT
#% key: elevation
#%end
#%option G_OPT_V_INPUT
#% key: points
#%end

import grass.script as gscript

def main():
    options, flags = gscript.parser()
    elevation = options['elevation']
    points = options['points']
    print(elevation, points)
if __name__ == '__main__':
    main()



Overwriting r.viewshed.points.py


In [19]:
!./r.viewshed.points.py --help

Compute and analyze viewsheds

Usage:
 r.viewshed.points.py elevation=name points=name [--help] [--verbose]
   [--quiet] [--ui]

Flags:

Parameters:
  elevation   Name of input raster map
     points   Name of input vector map


In [20]:
!./r.viewshed.points.py elevation=elevation points=bridges

('elevation', 'bridges')


### 将参数添加为自定义选项

当使用标准选项作为参数时，所有的选项被解析器预定义，例如`key`。但是，当需要其他参数作为输入时，也可以自定义参数选项的信息。

在本示例中，引入了一个名为`max_distance`的非预定义关键字，其类型设置为`double`，使用`lable` 描述该选项的含义进行简短描述，使用`description`做更长描述，`key_desc`对正确语法进行简短提示，更多信息可以参考[existing GRASS GIS Python scripts](https://trac.osgeo.org/grass/browser/grass/branches/releasebranch_7_0/scripts/)。


`max_distance`参数默认值为`-1`，表示视域中的*infinity*。因为不需要用户提供此选项，因此将`required`设置为`no`。同样，此选项只能提供一个值，因此将`multiple`设置为`no`。

In [21]:
%%file r.viewshed.points.py
#!/usr/bin/env python

#%module
#% description: Compute and analyze viewsheds
#% keyword: raster
#% keyword: viewshed
#%end
#%option G_OPT_R_INPUT
#% key: elevation
#%end
#%option G_OPT_V_INPUT
#% key: points
#%end
#%option
#% key: max_distance
#% key_desc: value
#% type: double
#% description: Maximum visibility radius. By default infinity (-1)
#% answer: -1
#% multiple: no
#% required: no
#%end

import grass.script as gscript

def main():
    options, flags = gscript.parser()
    elevation = options['elevation']
    points = options['points']
    max_distance = float(options['max_distance'])
    print(elevation, points, max_distance)

if __name__ == '__main__':
    main()



Overwriting r.viewshed.points.py


In [22]:
!./r.viewshed.points.py elevation=elevation points=bridges max_distance=50

('elevation', 'bridges', 50.0)


### 添加flags参数

In [23]:
%%file r.viewshed.points.py
#!/usr/bin/env python

#%module
#% description: Compute and analyze viewsheds
#% keyword: raster
#% keyword: viewshed
#%end
#%option G_OPT_R_INPUT
#% key: elevation
#%end
#%option G_OPT_V_INPUT
#% key: points
#%end
#%option
#% key: max_distance
#% key_desc: value
#% type: double
#% description: Maximum visibility radius. By default infinity (-1)
#% answer: -1
#% multiple: no
#% required: no
#%end
#%flag
#% key: c
#% description: Consider the curvature of the earth (current ellipsoid)
#%end


import grass.script as gscript

def main():
    options, flags = gscript.parser()
    elevation = options['elevation']
    points = options['points']
    max_distance = float(options['max_distance'])
    use_curvature = flags['c']
    print(elevation, points, max_distance)
    if use_curvature:
        print(use_curvature, "is true")
    else:
        print(use_curvature, "is false")

if __name__ == '__main__':
    main()



Overwriting r.viewshed.points.py


In [24]:
!./r.viewshed.points.py -c elevation=elevation points=bridges max_distance=50

('elevation', 'bridges', 50.0)
(True, 'is true')


## 编译的形式要求

如前所述，GRASS GIS模块必须使用GRASS解析器来处理其命令行参数，模块以GRASS栅格或矢量地图的形式读取和/或写入地理空间数据。

除非用户使用`--overwrite`标志指定，否则模块不能覆盖现有数据，对于栅格和矢量地图以及其他文件，如果输出已经存在，则GRASS解析器会自动检查它们的存在并以适当的错误消息结束模块执行。


另外，必须为每一个模块提供一个注释文本，利用[simple HTML syntax](https://trac.osgeo.org/grass/wiki/Submitting/Docs#HTMLPages)进行编写。


该文档随附件一起插件，并且也可以自动在线获得([GRASS addons docs](http://grass.osgeo.org/grass70/manuals/addons/))，文档的基本结构入下节所示：


In [25]:
%%file r.viewshed.points.html
<h2>DESCRIPTION</h2>

A long description with details about the method, implementation, usage or whatever is appropriate.

<h2>EXAMPLES</h2>

Examples of how the module can be used alone or in combination with other modules.
Possibly using the GRASS North Carolina State Plane Metric sample Location.
At least one screenshot (PNG format) of the result should provided to show the user what to expect.

<h2>REFERENCES</h2>

Reference or references to paper related to the module or references which algorithm the module is based on.

<h2>SEE ALSO</h2>

List of related or similar GRASS GIS modules or modules used together with this module as well as any related websites, or
related pages at the GRASS GIS User wiki.

<h2>AUTHORS</h2>

List of author(s), their organizations and funding sources.

Writing r.viewshed.points.html


所有GRASS GIS模块必须根据GNU GPL许可证（版本2或更高版本）进行分发。

以下是一种指定的模块主文件注释模板：



    """
    MODULE:    r.viewshed.points

    AUTHOR(S): jiangroubao <dxdkx@cau.edu.cn>

    PURPOSE:   Computes viewshed for multiple points

    COPYRIGHT: (C) 2020 jiangroubao

               This program is free software under the GNU General Public
               License (>=v2). Read the file COPYING that comes with GRASS
               for details.
    """

尽管Python不是像C一样的编译语言，但我们还需要“编译”基于Python的GRASS GIS模块，以便将其包含在GRASS安装中，并创建HTML文档和GUI。 为此，需要编写一个*Makefile*，该文件也要遵循标准模板。假定Python脚本，HTML文档和PNG格式的图片位于同一目录中，则包含的`Script.make`将处理所有内容。


In [26]:
%%file Makefile
MODULE_TOPDIR = ../..

PGM = r.viewshed.points

include $(MODULE_TOPDIR)/include/Make/Script.make

default: script

Writing Makefile


No we will compile the module which will also add it to our GRASS GIS installation. To do this, we need to have administrator rights (on Linux, use sudo in command line). First we need to create one directory required for the compilation:


编译该模块并将其添加到GRASS GIS安装中,为此，需要具有管理员权限（在Linux上，在命令行中使用sudo）,首先，我们需要创建一个编译所需的目录：

In [ ]:
!mkdir `grass70 --config path`/scriptstrings

然后可以进行编译

In [ ]:
!make MODULE_TOPDIR=`grass70 --config path`

在这两个命令中，我们使用“反引号”语法将命令的输出包含到另一个命令中。

## 对临时地图和计算区域的处理

在脚本中，我们通常需要创建临时的图层，脚本完成后或出现错误时需要将其删除。

同样，当需要在脚本中更改计算区域时，使用临时区域，这样就不会影响当前的区域设置，这也允许并行运行多个脚本，每个脚本都有自己的区域设置。



### 临时地图图层

下面将通过简单的示例脚本展示如何处理临时图层。

通过生成临时随机图，每个图都有唯一的名称，可以通过将进程ID与其他一些前缀结合使用来获得唯一名称：




In [45]:
%%python
import os
import grass.script as gscript

def main():
    for i in range(5):
        name = 'tmp_raster_' + str(os.getpid()) + str(i)
        gscript.mapcalc("{name} = rand(0, 10)".format(name=name), seed=1)

if __name__ == '__main__':
    main()

为了简化手动删除临时地图步骤，使用[g.remove](http://grass.osgeo.org/grass70/manuals/g.remove.html)进行批量删除

In [57]:
gscript.run_command('g.remove', type='raster', pattern="tmp_raster_*", flags='f')

0

在下面的脚本末尾编写了一个删除这些临时图层的函数。

In [37]:
%%python
import os
import grass.script as gscript

TMP_MAPS = []

def main():
    for i in range(5):
        name = 'tmp_raster_' + str(os.getpid()) + str(i)
        TMP_MAPS.append(name)
        gscript.mapcalc("{name} = rand(0, 10)".format(name=name), seed=1)
    cleanup()
        
def cleanup():
    gscript.run_command('g.remove', type='raster', name=','.join(TMP_MAPS), flags='f')

if __name__ == '__main__':
    main()


GRASS_INFO_MESSAGE(126457,1): Removing raster <tmp_raster_1264470>
GRASS_INFO_END(126457,1)

GRASS_INFO_MESSAGE(126457,2): Removing raster <tmp_raster_1264471>
GRASS_INFO_END(126457,2)

GRASS_INFO_MESSAGE(126457,3): Removing raster <tmp_raster_1264472>
GRASS_INFO_END(126457,3)

GRASS_INFO_MESSAGE(126457,4): Removing raster <tmp_raster_1264473>
GRASS_INFO_END(126457,4)

GRASS_INFO_MESSAGE(126457,5): Removing raster <tmp_raster_1264474>
GRASS_INFO_END(126457,5)


但是，如果发生错误，将不会调用cleanup函数：

In [55]:
%%python
import os
import grass.script as gscript

TMP_MAPS = []

def main():
    for i in range(5):
        name = 'tmp_raster_' + str(os.getpid()) + str(i)
        TMP_MAPS.append(name)
        gscript.mapcalc("{name} = rand(0, 10)".format(name=name), seed=1)
    # we raise intentionally error here:
    raise TypeError
    cleanup()

def cleanup():
    gscript.run_command('g.remove', type='raster', name=','.join(TMP_MAPS), flags='f')

if __name__ == '__main__':
    main()

Traceback (most recent call last):
  File "<stdin>", line 19, in <module>
  File "<stdin>", line 12, in main
TypeError


In [60]:
gscript.list_pairs(type='raster', pattern='tmp_raster_*')

[]

Therefore we will use Python _atexit_ module to ensure the cleanup functions is called everytime, even when the script ends with error.

因此，使用Python _atexit_ 模块，确保清除函数每次都被调用，即使遇到了程序错误

In [59]:
%%python
import os
import atexit
import grass.script as gscript

TMP_MAPS = []

def main():
    for i in range(5):
        name = 'tmp_raster_' + str(os.getpid()) + str(i)
        TMP_MAPS.append(name)
        gscript.mapcalc("{name} = rand(0, 10)".format(name=name), seed=1)
    # we raise intentionally error here:
    raise TypeError

def cleanup():
    gscript.run_command('g.remove', type='raster', name=','.join(TMP_MAPS), flags='f')

if __name__ == '__main__':
    atexit.register(cleanup)
    main()

Traceback (most recent call last):
  File "<stdin>", line 20, in <module>
  File "<stdin>", line 13, in main
TypeError

GRASS_INFO_MESSAGE(127974,1): Removing raster <tmp_raster_1279570>
GRASS_INFO_END(127974,1)

GRASS_INFO_MESSAGE(127974,2): Removing raster <tmp_raster_1279571>
GRASS_INFO_END(127974,2)

GRASS_INFO_MESSAGE(127974,3): Removing raster <tmp_raster_1279572>
GRASS_INFO_END(127974,3)

GRASS_INFO_MESSAGE(127974,4): Removing raster <tmp_raster_1279573>
GRASS_INFO_END(127974,4)

GRASS_INFO_MESSAGE(127974,5): Removing raster <tmp_raster_1279574>
GRASS_INFO_END(127974,5)


### 临时计算区域

使用临时计算区域的方法也很简单，主要是在脚本开始时调用`use_temp_region` ，然后可以在脚本中调用[g.region](http://grass.osgeo.org/grass70/manuals/g.region.html) 。

首先，设置符合栅格图像的计算区域。

In [61]:
import grass.script as gscript
gscript.run_command('g.region', raster='elevation')
print gscript.region()

{u'rows': 1350, u'e': 645000.0, u'projection': 99, u'zone': 0, u'cells': 2025000, u'cols': 1500, u'n': 228500.0, u's': 215000.0, u'w': 630000.0, u'ewres': 10.0, u'nsres': 10.0}


如本示例所示，脚本中设置的区域不会影响上一步中设置的当前区域。

In [62]:
%%python
import grass.script as gscript


def main():
    gscript.run_command('g.region', n=100, s=0, w=0, e=100, res=1)
    print gscript.region()

if __name__ == '__main__':
    gscript.use_temp_region()
    main()

{u'rows': 100, u'e': 100.0, u'projection': 99, u'zone': 0, u'cells': 10000, u'cols': 100, u'n': 100.0, u's': 0.0, u'w': 0.0, u'ewres': 1.0, u'nsres': 1.0}


In [64]:
print gscript.region()# 提前设置的区域并没有被改变

{u'rows': 1350, u'e': 645000.0, u'projection': 99, u'zone': 0, u'cells': 2025000, u'cols': 1500, u'n': 228500.0, u's': 215000.0, u'w': 630000.0, u'ewres': 10.0, u'nsres': 10.0}


## 完整的GRASS GIS 的示例

现在，把上一节中的脚本转换成一个完整的模块。

该模块的输入是带有点的矢量图和高程栅格图。输出是带有视域特征的矢量点的新矢量地图，该视域特征保存在属性表和栅格视域中（基名称在G_OPT_R_BASENAME_OUTPUT中指定）。此外，该模块允许设置最大距离和观察者仰角，并在计算视域时考虑曲率。


__注意:__ 使用 `r.viewshed --script`获取 `max_distance` 和 `observer_elevation`选项的定义。




In [65]:
%%file r.viewshed.points.py
#!/usr/bin/env python

#%module
#% description: Compute and analyze viewsheds
#% keyword: raster
#% keyword: viewshed
#%end
#%option G_OPT_R_INPUT
#% key: elevation
#%end
#%option G_OPT_V_INPUT
#% key: points
#%end
#%option G_OPT_V_OUTPUT
#% key: output_points
#%end
#%option G_OPT_R_BASENAME_OUTPUT
#% key: viewshed_basename
#%end
#%option
#% key: observer_elevation
#% type: double
#% required: no
#% multiple: no
#% key_desc: value
#% description: Viewing elevation above the ground
#% answer: 1.75
#%end
#%option
#% key: max_distance
#% key_desc: value
#% type: double
#% description: Maximum visibility radius. By default infinity (-1)
#% answer: -1
#% multiple: no
#% required: no
#%end
#%flag
#% key: c
#% description: Consider the curvature of the earth (current ellipsoid)
#%end


import os
import atexit
import grass.script as gscript
from grass.pygrass.vector import VectorTopo


TMP_MAPS = []


def main():
    options, flags = gscript.parser()
    elevation = options['elevation']
    input_points = options['points']
    basename = options['viewshed_basename']
    output_points = options['output_points']
    observer_elev = options['observer_elevation']
    max_distance = float(options['max_distance'])
    flag_curvature = 'c' if flags['c'] else ''

    tmp_prefix = 'tmp_r_viewshed_points_' + str(os.getpid()) + '_'
    tmp_viewshed_name = tmp_prefix + 'viewshed'
    tmp_viewshed_vector_name = tmp_prefix + 'viewshed_vector'
    tmp_visible_points = tmp_prefix + 'points'
    tmp_point = tmp_prefix + 'current_point'
    TMP_MAPS.extend([tmp_viewshed_name, tmp_viewshed_vector_name, tmp_visible_points, tmp_point])

    columns = [('cat', 'INTEGER'),
               ('area', 'DOUBLE PRECISION'),
               ('n_points_visible', 'INTEGER'),
               ('distance_to_closest', 'DOUBLE PRECISION')]

    with VectorTopo(input_points, mode='r') as points, \
         VectorTopo(output_points, mode='w', tab_cols=columns) as output:

        for point in points:
            viewshed_id = str(point.cat)
            viewshed_name = basename + '_' + viewshed_id
            gscript.run_command('r.viewshed', input=elevation,
                                output=tmp_viewshed_name, coordinates=point.coords(),
                                max_distance=max_distance, flags=flag_curvature,
                                observer_elev=observer_elev, overwrite=True, quiet=True)
            gscript.mapcalc(exp="{viewshed} = if({tmp}, {vid}, null())".format(viewshed=viewshed_name,
                                                                               tmp=tmp_viewshed_name,
                                                                               vid=viewshed_id))

            # viewshed size
            cells = gscript.parse_command('r.univar', map=viewshed_name,
                                          flags='g')['n']
            area = float(cells) * gscript.region()['nsres'] * gscript.region()['nsres']

            # visible points
            gscript.run_command('r.to.vect', input=viewshed_name, output=tmp_viewshed_vector_name,
                                type='area', overwrite=True, quiet=True)
            gscript.run_command('v.select', ainput=input_points, atype='point',
                                binput=tmp_viewshed_vector_name, btype='area', 
                                operator='overlap', flags='t', 
                                output=tmp_visible_points, overwrite=True, quiet=True)
            n_points_visible = gscript.vector_info_topo(tmp_visible_points)['points'] - 1

            # distance to closest visible point
            if float(n_points_visible) >= 1:
                gscript.write_command('v.in.ascii', input='-', stdin='%s|%s' % (point.x, point.y),
                                      output=tmp_point, overwrite=True, quiet=True)
                distance = gscript.read_command('v.distance', from_=tmp_point, from_type='point', flags='p',
                                                to=tmp_visible_points, to_type='point', upload='dist',
                                                dmin=1, quiet=True).strip()

                distance = float(distance.splitlines()[1].split('|')[1])
            else:
                distance = 0

            #
            # write each point with its attributes
            #
            output.write(point, (area, n_points_visible, distance))
            output.table.conn.commit()

def cleanup():
    gscript.run_command('g.remove', type='raster', name=','.join(TMP_MAPS), flags='f')
    
if __name__ == '__main__':
    atexit.register(cleanup)
    main()



Overwriting r.viewshed.points.py


In [67]:
!sudo make MODULE_TOPDIR=`grass76 --config path`

[sudo] password for geo: 


In [ ]:
#调用编译到GRASS中的自定义模块
import grass.script as gscript
gscript.run_command('r.viewshed.points', elevation='elevation',
                    points='input_points', viewshed_basename='viewshed',
                    output_points='new_points', flags='c', overwrite=True)

## 将新的模块插件上传到GRASS GIS 插件项目中

It is important to follow the __PEP8 style__ guide. You can use the *pep8* tool to control it. To identify problems in your Python code, you can use *pylint*. See also the GRASS GIS [submitting rules and best practices](https://trac.osgeo.org/grass/wiki/Submitting/Python) for Python code. Please note that not all examples here or elsewhere strictly follow all PEP8 rules. This is usually just for saving space or for other documentation-related reasons. Another case when PEP8 cannot be applied are interface definition comments (`#%`) at the beginning of the file.  However in general, you should always respect PEP8 in your scripts.

Read about getting a write access to GRASS addons repository at [How To Contribute](https://trac.osgeo.org/grass/wiki/HowToContribute#WriteaccesstotheGRASS-Addons-SVNrepository) and then contact the GRASS Project Steering Committee, the [grass-psc](https://lists.osgeo.org/listinfo/grass-psc) mailing list (you have to subscribed to the list first).

GRASS GIS is using a Subversion (SVN) repository provided by OSGeo. To get your code into the repository upon acceptance by the PSC, you need the following commands:
* `svn checkout https://svn.osgeo.org/grass/grass-addons` to download locally the repository,
* `svn add` to add your files and directories; all files for one module should be in one directory named according to the module, so in our case `r.viewshed.points` directory, put this directory to the proper subdirectory of `grass7` directory according to module's category, e.g. `r.viewshed.points` module shoud go to `grass7/raster`,
* set the SVN file properties locally which is easily done with is a script (`tools/module_svn_propset.sh`) found in the `grass-addons` (this script, provided with the files to update, will set the required properties automatically),
* `svn commit` to commit a new version and upload it to the remote SVN repository.

Now the new module is available to all users and can be installed with [g.extension](http://grass.osgeo.org/grass70/manuals/g.extension.html). Furthermore, it will show up in the subsequent day(s) in the [GRASS GIS 7 addon manual pages](http://grass.osgeo.org/grass70/manuals/addons/).